In [1]:
import gradio as gr
import os
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils
import numpy as np

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')
root_path = PATH + '/Datasets/MeDAL'
data_path = root_path + '/training_data(2).jsonl'

In [33]:
from scipy.io import wavfile

# np.load('Datasets/Audio_Files/Talha Ahmed_23_M/Talha Ahmed_23_M_Query_4.wav', allow_pickle=True)
audio = wavfile.read('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files/Talha Ahmed_23_M/Talha Ahmed_23_M_Query_4.wav', )

In [40]:
import os

# File name with old extension
old_filename = 'C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files/Talha Ahmed_23_M/Talha Ahmed_23_M_Query_4.wav'

# Extract the file name without extension
filename_without_extension = os.path.splitext(old_filename)[0]

# New filename with .mp3 extension
new_filename = filename_without_extension + '.mp3'

# Rename the file
os.rename(old_filename, new_filename)

In [75]:
from pydub import AudioSegment
audio = AudioSegment.from_mp3('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files/Talha Ahmed_23_M/Talha Ahmed_23_M_Query_4.mp3')

audio

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [72]:
import librosa
y, sr = librosa.load('C:/Users/Talha/OneDrive - Higher Education Commission/Documents/GitHub/CS-5302-Project-Group-15/Datasets/Audio_Files/Talha Ahmed_23_M/Talha Ahmed_23_M_Query_4.mp3', sr = 16000)

In [74]:
# convert to expected format
if audio_segment.frame_rate != 16000: # 16 kHz
    audio_segment = audio_segment.set_frame_rate(16000)
if audio_segment.sample_width != 2:   # int16
    audio_segment = audio_segment.set_sample_width(2)
if audio_segment.channels != 1:       # mono
    audio_segment = audio_segment.set_channels(1)        
arr = np.array(audio_segment.get_array_of_samples())
arr = arr.astype(np.float32)/32768.0

result = audio_model.transcribe(arr, language=language, fp16=torch.cuda.is_available())
print(result['text'])

16000

In [ ]:
def preprocess_audio(audio):
    if audio[0] != 16000: # 16 kHz
        audio = audio.set_frame_rate(16000)
    if audio.sample_width != 2:   # int16
        audio = audio.set_sample_width(2)
    if audio.channels != 1:       # mono
        audio = audio.set_channels(1)        
    arr = np.array(audio.get_array_of_samples())
    arr = arr.astype(np.float32)/32768.0

    result = audio_model.transcribe(arr, language=language, fp16=torch.cuda.is_available())
    print(result['text'])

In [70]:
def SMTS(audio):
    try:    
        # Step 2: Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        # audio_filter = audio[1].astype(np.float32)/32768.0
        transcript = whisper_setup.transcribe_audio(audio_filter, ['tiny'])

        # Step 3: Feed query into the LLM
        models = { 
            'llama_ours': 'ubaidtariq8/llama2-med-genai', # fine tuned model
            'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
            'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
            'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }
        
        print("0")
        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['llama_ours'], chroma_path = root_path)
        print("1")
        model.setup_environment()
        print("2")
        model.prepare_documents(data_path = data_path, collection_name = "medal_filtered", joining = True, persistent = False)
        print("3")
        model.embed_and_index()
        print("4")
        response = [model.query_data(query = transcript['tiny'])]
        print("5")

        # Step 4: Translate it back to the user language
        translated_text = machine_translation.translate_text(text = " ".join(response), src_lang = 'en', trg_lang = transcript['lang'])

        # Step 5: Now speak the response in the user's language
        # answer_path = utils.generate_answer_path(original_path = audio_path)
        audio_answer_path = data_path + '/audio.wav'
        text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = audio_answer_path)

        # display(Markdown(f"<b>{translated_text}</b>"))
        return translated_text, utils.play_wav(audio_answer_path)
    except Exception as e:
        print("An error occurred:", e)

In [71]:
demo = gr.Interface(
    SMTS,
    gr.Audio(sources = ["microphone"]),
    ["text", "audio"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


[  0   0   0 ... -77 -45 -86]
An error occurred: cannot access local variable 'text' where it is not associated with a value


Traceback (most recent call last):
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1797, in process_api
    data = await self.postprocess_data(fn_index, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1565, in postprocess_data
    self.validate_outputs(fn_index, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^